In [7]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
import pickle
import copy

import pandas as pd
import seaborn as sns
import numpy
import torch
import scipy
import scipy.stats
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import os
dim=2
base_dir='/Users/phumon/Documents/Research/nlu-test-sets/params_dim3'
file_name=f'alpha-lognormal-identity-dim3_theta-normal-identity_nosubsample_1.00_0.30'
exp_dir=os.path.join(base_dir, file_name)

print(exp_dir)


/Users/phumon/Documents/Research/nlu-test-sets/params_dim3/alpha-lognormal-identity-dim3_theta-normal-identity_nosubsample_1.00_0.30


In [3]:
param_plot_stats = {}
keys = ['a', 'b', 'g', 't']
for key in keys:
    with open(os.path.join(exp_dir, 'plot_stats_pickles', f'{key}.p'), 'rb') as f:
        param_plot_stats[key] = pickle.load(f)

In [4]:

param_plot_stats['a']['log_mean'] = param_plot_stats['a']['mean'].apply(lambda x: numpy.log(x))
alpha_mean, alpha_var = param_plot_stats['a']["log_mean"].to_numpy().mean(), param_plot_stats['a']["var"].to_numpy().mean()
print(alpha_mean, alpha_var)

[-0.0669063  -0.06938468 -0.06983731] [0.0667566  0.06635319 0.06628769]


In [8]:
dimensions=3
for dim in range(dimensions):
    param_plot_stats['a']['log_mean_'+str(dim)]= param_plot_stats['a']['log_mean'].apply(lambda x: x[dim])
    param_plot_stats['b']['mean_'+str(dim)]= param_plot_stats['b']['mean'].apply(lambda x: x[dim])

In [9]:
plt.figure()
for dim in range(dimensions):
    plt.plot(param_plot_stats['a']['log_mean_'+str(dim)], param_plot_stats['b']['mean_'+str(dim)], marker="o", linestyle="", label=dim)

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plt.figure()
plt.plot(param_plot_stats['a']['log_mean_0'], param_plot_stats['a']['log_mean_1'], marker="o", linestyle="", label=1)
plt.plot(param_plot_stats['a']['log_mean_0'], param_plot_stats['a']['log_mean_2'], marker="o", linestyle="", label=2)

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
param_plot_stats['a']['log_mean']

0       [-0.05879890045855681, -0.051111937036824116, ...
1       [-0.10252156667959851, -0.13336492219368679, -...
2       [-0.09378338683776374, -0.0946903584578507, -0...
3       [-0.08367149278981982, -0.09039450925051422, -...
4       [-0.02870992996194188, -0.02311715793477296, -...
                              ...                        
9819    [-0.11391405977598786, -0.13146786459437604, -...
9820    [-0.13125762711599462, -0.1210941252630924, -0...
9821    [-0.10658537270152897, -0.10913480181887919, -...
9822    [-0.07193894732048856, -0.05841102880880827, -...
9823    [-0.08974704564706987, -0.0899160618920721, -0...
Name: log_mean, Length: 9824, dtype: object

In [30]:
plt.figure()
plt.plot(param_plot_stats['a']['log_mean_0'], param_plot_stats['a']['log_mean_2'], marker="o", linestyle="")
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [38]:
plt.figure()
plt.plot(param_plot_stats['b']['mean_0'], param_plot_stats['b']['mean_1'], marker="o", linestyle="", label=1)
plt.plot(param_plot_stats['b']['mean_0'], param_plot_stats['b']['mean_2'], marker="o", linestyle="", label=2)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

https://stackoverflow.com/questions/35407669/independent-component-analysis-ica-in-python

In [39]:
alpha_arr = np.vstack(param_plot_stats['b']['mean'])
pca = PCA(n_components=2, random_state=0)
new_alpha_ = pca.fit_transform(alpha_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [40]:
print(new_alpha_.shape)

(9824, 2)


In [41]:
from multi_virt_v2 import *

In [42]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

In [43]:
#datasets="boolq,cb,commonsenseqa,copa,cosmosqa,hellaswag,rte,snli,wic,qamr,arct,mcscript,mctaco,mutual,mutual-plus,quoref,socialiqa,squad-v2,wsc,mnli,mrqa-nq,newsqa,abductive-nli,arc-easy,arc-challenge,piqa,quail,winogrande,anli"
datasets="snli"
data_names, responses, n_items = get_files(
    os.path.join(repo, 'data'),
    "csv",
    set(datasets.split(','))
)

In [44]:
task_metadata = pd.read_csv('../irt_scripts/task_metadata.csv')
task_metadata.set_index("jiant_name", inplace=True)
task_list = [x for x in task_metadata.index if x in data_names]

In [45]:
total = 0
task_name = []
task_format = []

for tname, size in zip(data_names, n_items):
    name = task_metadata.loc[tname]['taskname']
    total += size
    task_name += [name for _ in range(size)]
    task_format += [task_metadata.loc[tname]['format'] for _ in range(size)]
    
task_name = pd.DataFrame(task_name, columns=['task_name'])
task_format = pd.DataFrame(task_format, columns=['format'])
task_name_format = pd.concat([task_name, task_format], axis=1)

In [46]:
task_name_format

,task_name,format
0,SNLI,classification
1,SNLI,classification
2,SNLI,classification
3,SNLI,classification
4,SNLI,classification
...,...,...
9819,SNLI,classification
9820,SNLI,classification
9821,SNLI,classification
9822,SNLI,classification


In [47]:

param = pd.concat([pd.DataFrame(new_alpha_.tolist()), task_name_format], axis=1)

In [48]:
param

,0,1,task_name,format
0,-1.673361,-0.006339,SNLI,classification
1,0.695956,0.018800,SNLI,classification
2,-0.338442,0.001636,SNLI,classification
3,-0.411508,0.002010,SNLI,classification
4,-1.747632,0.010975,SNLI,classification
...,...,...,...,...
9819,-0.070466,0.004077,SNLI,classification
9820,0.590853,-0.001523,SNLI,classification
9821,0.432930,-0.009376,SNLI,classification
9822,1.665839,0.010784,SNLI,classification


In [49]:


plt.figure()
plt.plot(param[0], param[1], "kx")
plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
groups = param.groupby("task_name")
groups

In [51]:
plt.figure()

groups = param.groupby("format")
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="")
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


name:  classification


In [52]:
param[0]

0      -1.673361
1       0.695956
2      -0.338442
3      -0.411508
4      -1.747632
          ...   
9819   -0.070466
9820    0.590853
9821    0.432930
9822    1.665839
9823   -0.338972
Name: 0, Length: 9824, dtype: float64

# Theta

In [20]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
pca = PCA(n_components=2, random_state=0)
new_beta_ = pca.fit_transform(beta_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [21]:
params_beta = pd.concat([pd.DataFrame(new_beta_.tolist()), task_name_format], axis=1)

In [22]:
groups = params_beta.groupby("task_name")


In [110]:
plt.figure()
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=name)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

name:  classification


In [111]:
# END scatter plot

In [66]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
ica = FastICA(n_components=3, random_state=0)
S_beta_ = ica.fit_transform(beta_arr)  # Reconstruct signals
A_beta_ = ica.mixing_  # Get estimated mixing matrix


In [67]:
models = [beta_arr, S_beta_]
names = ['Alpha (mixed signal)',
         'ICA recovered Alpha signals']
colors = ['red', 'steelblue', 'orange']

plt.figure()
for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(2, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()

/Users/phumon/opt/anaconda3/envs/latent_features/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
params_beta = pd.concat([pd.DataFrame(S_beta_.tolist()), task_name_format], axis=1)

In [69]:
params_beta

,0,1,2,task_name,format
0,-0.001264,-0.025934,0.002217,SNLI,classification
1,0.012072,0.008210,-0.010447,SNLI,classification
2,-0.006255,0.007303,-0.005794,SNLI,classification
3,-0.005112,0.004887,-0.005670,SNLI,classification
4,0.000617,-0.012774,-0.014000,SNLI,classification
...,...,...,...,...,...
9819,0.000196,0.002809,-0.004063,SNLI,classification
9820,0.005176,-0.002183,0.005583,SNLI,classification
9821,-0.003580,0.002087,0.008134,SNLI,classification
9822,0.030946,-0.017543,0.009977,SNLI,classification


In [70]:
groups = params_beta.groupby("task_name")
groups

In [71]:
plt.figure()
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=1)
    plt.plot(group[0], group[2], marker="o", linestyle="", label=2)
plt.legend()
plt.show()

/Users/phumon/opt/anaconda3/envs/latent_features/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

name:  SNLI
